In [1]:
import requests
import json
import pandas as pd
import numpy as np
import datetime
from config import api_key
from config import google_key
import time

import psycopg2
import sqlalchemy
import urllib
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import PrimaryKeyConstraint
from sqlalchemy.orm import Session
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, Integer, String, Float, DateTime
from sqlalchemy.schema import Sequence

print(f'Dependencies imported...')
print('---------------')

Dependencies imported...
---------------


In [2]:
data = []

headers = {'Authorization': 'Bearer %s' % api_key}

url='https://api.yelp.com/v3/businesses/search'

print('Downloading Yelp Data...')

for offset in range(0, 1000, 50):
    
    params = {
        'limit':50, 
        'location':'Minneapolis, MN',

        'categories':'restaurants',
        'offset':offset
        }  
    
    response=requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        data += response.json()['businesses']
    elif response.status_code == 400:
        print('400 Bad Request')
        break
        
print(f'Yelp data downloaded...  There are {len(data)} records...')
print('---------------')

Yelp data downloaded...  There are 1000 records...
---------------


In [3]:
i=0
yelp_list=[]
for places in data:
    yelp_id=data[i]['id']
    name=data[i]['name']
    image=data[i]['image_url']
    categories = []
    for category in data[i]['categories']:
        cat = category['title']
        categories.append(cat)
    url=data[i]['url']
    transactions=data[i]['transactions']
    city=data[i]['location']['city']
#     price=data[i]['price']
    phone=data[i]['display_phone']
    address= data[i]['location']['display_address']
    rating=data[i]['rating']
    reviews=data[i]['review_count']
    latitude=data[i]['coordinates']['latitude']
    longitude=data[i]['coordinates']['longitude']
    if data[i]['is_closed']==False and city=="Minneapolis":
        business_dict={"yelpid":yelp_id,"name":name,"image":image,"url":url,"latitude":latitude,"longitude":longitude,"phone":phone,"categories":categories,"transactions":transactions,"address":' '.join(map(str, address)),"rating":rating,"reviews":reviews}
        yelp_list.append(business_dict)
    i+=1

print('yelp_list with needed data has been built.')
print('---------------')

yelp_list with needed data has been built.
---------------


In [4]:
yelp_df=pd.DataFrame(yelp_list)
yelp_df=yelp_df[['yelpid','name','image','url','latitude','longitude','address','phone','categories','transactions','rating','reviews']]
yelp_df = yelp_df.drop_duplicates(subset=['name','address'])
yelp_df.to_csv("DataFiles/YelpData.csv")
print('Yelp DataFrame now stored in memory as "yelp_df" and csv "YelpData.csv" has been saved in DataFiles folder.')
print(f'Removed duplicates and restaurants outside of Minneapolis. Leaving {len(yelp_df)} restaurants.')
print('---------------')

Yelp DataFrame now stored in memory as "yelp_df" and csv "YelpData.csv" has been saved in DataFiles folder.
Removed duplicates and restaurants outside of Minneapolis. Leaving 817 restaurants.
---------------


In [5]:
print('Matching Yelp data list to Google API...   This will take some time, as we match each record...')

url = 'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?'
google_data=[]

for index,row in yelp_df.iterrows():
    
    params = {
        'key':google_key,
        'input':row['name'],
        'inputtype':'textquery',
        'locationbias': 'point:' + str(row['latitude']) + ", " + str(row['longitude']),
        'radius': 10,
        'fields':'place_id,name,formatted_address,geometry,rating,user_ratings_total,price_level,photos,icon'
        }
    
    response = requests.get(url, params=params)

    if len(response.json()['candidates'])>0:
        google_data.append(response.json()['candidates'][0])
    else:
        google_data.append("")
        
    print("Restaurants Remaining: {:3}".format(len(yelp_df)-index), end="\r")

print(f'Google match has been completed...  There are {len(google_data)} records')

Matching Yelp data list to Google API...   This will take some time, as we match each record...


KeyboardInterrupt: 

In [6]:
google_data


[{'formatted_address': '1121 Hennepin Ave, Minneapolis, MN 55403, United States',
  'geometry': {'location': {'lat': 44.9747894, 'lng': -93.27982670000002},
   'viewport': {'northeast': {'lat': 44.97610247989272,
     'lng': -93.27857022010728},
    'southwest': {'lat': 44.97340282010727, 'lng': -93.28126987989272}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/restaurant-71.png',
  'name': 'Butcher & The Boar',
  'photos': [{'height': 675,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/109104606351586527560">Butcher &amp; the Boar</a>'],
    'photo_reference': 'CmRaAAAA37fZ2ZHnAmxDd7VC9Sz2A_AB65VikqYhlNDa85uppMNL6vztnBQt8SfCT1Mv4R8Z3zsy3HEv-FHuRfVNtrwTK4nVmFkwi7NGvm5L_sYuurLdlD5XSdM4stJ0OEQP5P39EhDtLe5wWsgjr7Rk0Voa2EinGhT3NKBy-N4p8Rc5ZMkT8KW4cB53Sw',
    'width': 900}],
  'place_id': 'ChIJN2-3wpQys1IRK9G73sv97zg',
  'price_level': 3,
  'rating': 4.6,
  'user_ratings_total': 2110},
 {'formatted_address': '800 N Washington Ave, Minneapolis, MN 

In [9]:
i=0
google_list=[]

for places in google_data:
    if places != "":
        if "place_id" in places:
            google_id = places['place_id']
        if "icon"in places:
            icon=places['icon']
        photos=[]
        if "photos" in places:
            for photo in places['photos']:
                item = photo['html_attributions']
                photos.append(item)
        if "price_level" in places:
            price_level=places['price_level']
        if "name" in places:
            name = places['name']
        if "formatted_address" in places:
            address = places['formatted_address']
        if "rating" in places:
            rating  = places['rating']
        if "user_ratings_total" in places:
            reviews = places['user_ratings_total']
        if "geometry" in places:
            latitude = places['geometry']['location']['lat']
            longitude = places['geometry']['location']['lng']
        business_dict = {"googleplacesid":google_id,"icon":icon,"photos":photos,"name":name,"latitude":latitude,"longitude":longitude,"address":address,"rating":rating,"reviews":reviews,"price":price_level}
    
    else:
        business_dict = {"googleplacesid":"","icon":"","photos":"","name":"","latitude":"","longitude":"","address":"", "rating":"","reviews":"","price":""}
    
    google_list.append(business_dict)
    
    i+=1
    
print('google_list with needed data has been built.')
google_list

google_list with needed data has been built.


[{'googleplacesid': 'ChIJN2-3wpQys1IRK9G73sv97zg',
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/restaurant-71.png',
  'photos': [['<a href="https://maps.google.com/maps/contrib/109104606351586527560">Butcher &amp; the Boar</a>']],
  'name': 'Butcher & The Boar',
  'latitude': 44.9747894,
  'longitude': -93.27982670000002,
  'address': '1121 Hennepin Ave, Minneapolis, MN 55403',
  'rating': 4.6,
  'reviews': 2110,
  'price': 3},
 {'googleplacesid': 'ChIJBcDbP4oys1IRg97QGrkjtH8',
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/restaurant-71.png',
  'photos': [['<a href="https://maps.google.com/maps/contrib/103721415397737726598">Thomas Van Wylen</a>']],
  'name': 'Bar La Grassa',
  'latitude': 44.9894444,
  'longitude': -93.27861109999999,
  'address': '800 N Washington Ave, Minneapolis, MN 55401',
  'rating': 4.7,
  'reviews': 1428,
  'price': 3},
 {'googleplacesid': 'ChIJ6ZJ3YYUys1IRihzyYe0WiGs',
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/

In [ ]:
google_df=pd.DataFrame(google_list)
google_df=google_df[google_df.name != ""]
google_df = google_df.drop_duplicates(subset=['googleplacesid'])

google_df=google_df[['googleplacesid','name','latitude','longitude','address','rating','reviews','price','icon','photos']]

google_df.to_csv('DataFiles/googledata.csv')


print('Google DataFrame now stored in memory as "google_df" and csv "GoogleData.csv" has been saved in DataFiles folder.')
print(f'Removed null entries.  {len(google_df)} restaurants remain.')
print('---------------')

In [ ]:
i = 0
compare_list=[]
yelpgeo_list=[]

for i in range(len(google_list)):

    compare = {"Yelp":yelp_list[i]['name'],"Google":google_list[i]['name'],"GoogleAddress":google_list[i]['address'],"Yelp Address":yelp_list[i]['address']}
    compare_list.append(compare)
    i+=1

compare_df = pd.DataFrame(compare_list)
compare_df.to_csv('DataFiles/compare.csv')

print('"compare_df" has been stored in memory and csv "compare.csv" has been saved in DataFiles folder to allow easy comparison between Yelp and Google data.')
print('---------------')

In [ ]:
print('Matching Yelp data list to Minneapolis Health Inspection API...   This will take some time, as we match each record...')


inspection_data=[]

for index,row in yelp_df.iterrows():

    biz = row['name']

    biz_string = biz.split(' ',1)[0].upper()
    biz_string = biz_string.replace("'","")
    biz_string = biz_string.replace("&","")

    minlat=row['latitude']-.0015
    maxlat=row['latitude']+.0015
    minlon=row['longitude']-.0015
    maxlon=row['longitude']+.0015
    
    url = 'https://services.arcgis.com/afSMGVsC7QlRK1kZ/arcgis/rest/services/Food_Inspections/FeatureServer/0/query?'
    params = f"where=BusinessName%20like%20'%25{biz_string}%25'%20AND%20Latitude%20%3E%3D%20{minlat}%20AND%20Latitude%20%3C%3D%20{maxlat}%20AND%20Longitude%20%3E%3D%20{minlon}%20AND%20Longitude%20%3C%3D%20{maxlon}"
    outfields = "&outFields=BusinessName,HealthFacilityIDNumber,FullAddress,InspectionType,DateOfInspection,InspectionIDNumber,InspectionScore,Latitude,Longitude&returnGeometry=false&outSR=4326"
    json = '&f=json'

    full_url = url+params+outfields+json

    response = requests.get(full_url)
    
    if response !="":
        inspection_data += response.json()['features']
        
    print("Restaurants Remaining: {:3}".format(len(yelp_df)-index), end="\r")
    
print(f'Inspection data match has been completed...  There are {len(inspection_data)} records')
print('---------------')

In [ ]:
inspection_data_list = []

for records in inspection_data:
    item = records['attributes']
    item['DateOfInspection']=time.strftime('%m/%d/%Y',time.gmtime(records['attributes']['DateOfInspection']/1000))
    inspection_data_list.append(item)
    
print('inspection_data_list with needed data has been built.')
print('---------------')

In [ ]:
inspections_df1 = pd.DataFrame(inspection_data_list)
inspections_df1 = inspections_df1.drop_duplicates(subset='InspectionIDNumber', keep='first')
inspections_df = inspections_df1[['BusinessName','FullAddress',str('HealthFacilityIDNumber'),str('Latitude'),str('Longitude'),str('InspectionIDNumber'),str('DateOfInspection'),str('InspectionScore'),'InspectionType']]
inspections_df = inspections_df.rename(columns={'BusinessName':'businessname','FullAddress':'fulladdress','HealthFacilityIDNumber':'healthfacilityidnumber','Latitude':'latitude','Longitude':'longitude','InspectionIDNumber':'inspectionidnumber','DateOfInspection':'dateofinspection','InspectionScore':'inspectionscore','InspectionType':'inspectiontype'})

In [ ]:
inspect_by_biz=inspections_df.groupby(['businessname','fulladdress','latitude','longitude'],sort=False,as_index=False).aggregate(lambda x: list(x))

inspect_by_biz.to_csv('DataFiles/InspectionsData.csv')


print('Inspections DataFrame now stored in memory as "inspect_by_biz" and csv "InspectionsData.csv" has been saved in DataFiles folder.')
print(f'There are {len(inspections_df)} inspections for {len(inspect_by_biz)} facilities.')
print('---------------')

In [ ]:
#Postgres username, password, and database name
ipaddress = 'localhost'
port = '5432'
username = 'postgres'
password = 'password' 
dbname = 'Minneapolis_Restaurants'
# A long string that contains the necessary Postgres login information
postgres_str = f'postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'

In [ ]:
# Creates Classes which will serve as the anchor points for our Table, loads table to Postgres and uplads the data

Base = declarative_base()
engine = create_engine(postgres_str)

class YelpData(Base):
    __tablename__ = 'yelpdata'
    index=Column(Integer,primary_key=True,autoincrement=True)
    yelpid=Column(String,nullable=False)
    name=Column(String)
    image=Column(String)
    url=Column(String)
    latitude=Column(Float(20))
    longitude=Column(Float(20))
    address=Column(String)
    phone=Column(String)
    categories=Column(String)
    transactions=Column(String)
    rating=Column(Float(10))
    reviews=Column(Integer)
                   
Base.metadata.create_all(engine)

yelp_df.to_sql('yelpdata', engine, if_exists='replace', index=True)

print(f'Table "yelpdata" uploaded to postgreSQL database "Minneapolis_Restaurants".')
print('---------------')

In [ ]:
# Creates Classes which will serve as the anchor points for our Table, loads table to Postgres and uplads the data

Base = declarative_base()
engine = create_engine(postgres_str)

class GoogleData(Base):
    __tablename__ = 'googledata'
    googleplacesid=Column(String,primary_key=True, nullable=False)
    name=Column(String)
    latitude=Column(Float(20))
    longitude=Column(Float(20))
    address=Column(String)
    rating=Column(Float(10))
    reviews=Column(Integer) 
    price=Column(Integer)
    icon=Column(String)
    photos=Column(String)
                   
Base.metadata.create_all(engine)

google_df.to_sql('googledata', engine, if_exists='replace', index=True)

print(f'Table "googledata" uploaded to postgreSQL database "Minneapolis_Restaurants".')
print('---------------')

In [ ]:
# Creates Classes which will serve as the anchor points for our Table, loads table to Postgres and uplads the data

Base = declarative_base()
engine = create_engine(postgres_str)

class InspectionsData(Base):
    __tablename__ = 'inspectionsdata'
    index=Column(Integer,primary_key=True,autoincrement=True)
    businessname=Column(String,nullable=False)
    fulladdress=Column(String)
    healthfacilityidumber=Column(String)
    latitude=Column(Float(20))
    longitude=Column(Float(20))
    inspectionidnumber=Column(String)
    dateofinspection=Column(String)
    inspectionscore=Column(String)
    inspectiontype=Column(String)
                   
Base.metadata.create_all(engine)

inspect_by_biz.to_sql('inspectionsdata', engine, if_exists='replace', index=True)

print(f'Table "inspectionsdata" uploaded to postgreSQL database "Minneapolis_Restaurants".')
print('---------------')